In [4]:
import time
start_time = time.time()

In [5]:
from module.model import *
from module.train import *

from module.Environment import LEGOEnv
from module.ReplayBuffer import ReplayBuffer

import math, random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import pandas as pd
import copy
import os
from rich.console import Console

OSError: [WinError 126] 지정된 모듈을 찾을 수 없습니다

# 유전 알고리즘을 통해 얻은 submission들을 불러온다.

In [ ]:
console = Console()

path = "./RL_submission"
file_list = os.listdir(path)

genetic_submissions = []
name_dic = {}
idx = 0
for file in file_list:
    if 'submission' in file:
        submission = pd.read_csv(path+'/'+file)
        genetic_submissions.append(submission)
        name_dic[idx] = file
    idx += 1
    
# 860개의 csv파일 중에 100개만을 학습에 사용한다.
genetic_submissions = genetic_submissions[:100]

# 학습에 사용할 함수

* parameter으로 genetic_submission을 넣으면 해당 submission에 대한 transfer learning, 넣지 않으면 일반적인 RL학습  


* 가장 좋은 점수를 낸 environment와 iteration별 score을 리스트로 리턴한다.

In [ ]:
def training(epsilon_set, num_episodes=5000, genetic_submissions=None):
    
    epsilon_start = epsilon_set['epsilon_start']
    epsilon_final = epsilon_set['epsilon_final']
    epsilon_decay = epsilon_set['epsilon_decay']
    
    epsilon_by_frame = lambda frame_idx: epsilon_final + (epsilon_start - epsilon_final) * math.exp(
    -1. * frame_idx / epsilon_decay)
    
    best_episode_reward = 0
    
    if genetic_submissions is not None:
        num_episodes = len(genetic_submissions)
    
    epi_iter = 0
    
    score_l = []
    
    while epi_iter < num_episodes:
        
        if genetic_submissions is not None:
            submission = genetic_submissions[epi_iter]
            is_submission = True
        else:
            submission = None
        
        event_decisions = env.is_event_decision_time(submission)
        masks = env.get_avail_states_mask()

        epsilon = epsilon_by_frame(epi_iter)

        for line in [0, 1]:
            event_action = None
            type_action = None
            molding_action = None

            state = env.get_state()

            if event_decisions[line]:  # If this line needs a state-change decision
                event_mask = masks[line]
                event_action = net1.get_action(state, event_mask, epsilon, submission=submission, t=env.t, line=line)
                
                if submission is None:
                    # If the action is to continue PROCESS
                    if event_action == env.line_states[line] == 1:  # PROCESS
                        # Keep PROCESSing... by ignoring this action
                        event_action = None

                    # If the action is to continue STOP
                    if event_action == env.line_states[line] == 2:  # STOP
                        # Keep STOPing... by ignoring this action
                        event_action = None

                # If the next state is either CHECK or CHANGE, we need to determine the Brick type as well
                if event_action == 0 or event_action == 3:
                    type_mask = np.array([1, 1, 1, 1])
                    if event_action == 3:
                        # For the CHANGE, the type cannot be the same
                        type_mask[env.line_types[line] - 1] = 0
                    type_action = net2.get_action(state, type_mask, env.brick_mask, env.t,epsilon,submission=submission, t=env.t, line=line)

            # Molding amount action
            # Only do this if
            #   new event_action is PROCESS
            #              or
            #   this line is PROCESS and new event_action is PROCESS as well
            if env.t >= 23 * 24:  # The first 23 days have no molding operations
                if event_action == 1 or (env.line_states[line] == 1 and event_action is None):
                    molding_action, molding_action_index = net3.get_action(state, epsilon,submission=submission, t=env.t, line=line)

            if not (event_action is None and type_action is None and molding_action is None):
                next_state, reward_1, reward_2, reward_3, done = env.apply_action(line, event_action, type_action, molding_action, submission)
                if event_action is not None: 
                    replay_buffer1.push(state, event_action, reward_1, next_state, done, event_mask)

                if type_action is not None:
                    replay_buffer2.push(state, type_action-1, reward_2, next_state, done, type_mask)

                if molding_action is not None:
                    replay_buffer3.push(state, molding_action_index, reward_3, next_state, done, None)
        
        is_submission = False
        if submission is not None:
            is_submission = True
        state, done = env.preceed_time(is_submission)

        if done:

            epi_reward = env.get_reward()
            score_l.append(epi_reward)
            
            if submission is None:
                for line in [0, 1]:
                    replay_buffer1.push(state, env.line_states[line], epi_reward, next_state, done, event_mask)
                    replay_buffer2.push(state, env.line_types[line]-1, epi_reward, next_state, done, event_mask)
                    replay_buffer3.push(state, 0, epi_reward, next_state, done, None)
                        
            if epi_reward > best_episode_reward:
                best_env = copy.deepcopy(env)
                
            best_episode_reward = max(epi_reward, best_episode_reward)
            episode_rewards.append(epi_reward) 

            for list_event_action in env.tempbuffer_event_action:
                state, event_action, reward, next_state, done, event_mask = list_event_action

            for list_type_action in env.tempbuffer_type_action:
                state, type_action, reward, next_state, done, type_mask = list_type_action

            for list_molding_action in env.tempbuffer_molding_action:
                state, molding_action, reward, next_state, done = list_molding_action

            console.print(f'{epi_iter}: SCORE={epi_reward}, RL BEST={best_episode_reward}, e={epsilon:.2f}')

            state = env.reset()

            epi_iter += 1

            loss1 = train_net(replay_buffer1, net1, net1_target, optimizer1, batch_size, gamma, use_mask=False)
            loss2 = train_net(replay_buffer2, net2, net2_target, optimizer2, batch_size, gamma, use_mask=False)
            loss3 = train_net(replay_buffer3, net3, net3_target, optimizer3, batch_size, gamma, use_mask=False)

            loss1_l.append(loss1)
            loss2_l.append(loss2)
            loss3_l.append(loss3)
            
    return best_env, score_l


# Environment와 Event, Brick, Mold Amount에 대한 Network를 생성

In [ ]:
env = LEGOEnv()
state = env.reset()

len_state = len(state)

net1 = EventDecisionNN(len_state)
net2 = BrickTypeNN(len_state)
net3 = MoldingAmountNN(len_state)

net1_target = EventDecisionNN(len_state)
net1_target.load_state_dict(net1.state_dict())

net2_target = BrickTypeNN(len_state)
net2_target.load_state_dict(net2.state_dict())

net3_target = MoldingAmountNN(len_state)
net3_target.load_state_dict(net3.state_dict())

train_interval_target = 10

learning_rate = 0.05

optimizer1 = optim.Adam(net1.parameters(), lr=learning_rate)
optimizer2 = optim.Adam(net2.parameters(), lr=learning_rate)
optimizer3 = optim.Adam(net3.parameters(), lr=learning_rate)

num_episodes = 5000  # 1600
batch_size = 256
gamma = 0.99

sim_best_reward = 0

episode_rewards = []

best_episode_reward = 0

buffer_size = 500

# Replay buffer for EventDecisionNN
replay_buffer1 = ReplayBuffer(buffer_size * batch_size)

# Replay buffer for BrickTypeNN
replay_buffer2 = ReplayBuffer(buffer_size * batch_size)

# Replay buffer for EventDecisionNN
replay_buffer3 = ReplayBuffer(buffer_size * batch_size)

epi_iter = 0
    
loss1_l = []
loss2_l = []
loss3_l = []

# Action Transfer Learning (ATL)

submission들의 Action을 재현하여 buffer에 좋은 에피소드를 채워준다

In [ ]:
epsilon_set = {'epsilon_start':0, 'epsilon_final':0, 'epsilon_decay':1000}
ATL_best_env, ATL_score_l = training(epsilon_set, num_episodes=None, genetic_submissions=genetic_submissions)

# Train

일반적인 DDQN으로 강화학습을 진행

In [ ]:
epsilon_set = {'epsilon_start':1, 'epsilon_final':0.1, 'epsilon_decay':1000}
best_env, score_l = training(epsilon_set, num_episodes=100)

# Save Submission



In [ ]:
best_env.save_submission('submission_name.csv')

In [ ]:
time.time() - start_time